# **Welcome to The Notebook**

### Task 1 - Set up the project environment

Installing the needed modules

In [ ]:
!pip install openai==0.28 python-dotenv

Importing the modules

In [ ]:
import pandas as pd
import os
import openai
from dotenv import load_dotenv
import json

Setting up the OpenAI API:

1. Prepare a `.env` file to store the OpenAI API key.
2. Uploading the `.env` file to our colab environment
3. Load the API key and setup the API

Upload your `.env` file here

In [ ]:
from google.colab import files

uploaded = files.upload()

Now let's load environment variables and get the API key

In [ ]:
load_dotenv(dotenv_path="apikey.env.txt")

APIKEY = os.getenv("APIKEY")
ORGID = os.getenv("ORGID")

Let's setup our OpenAI API

In [ ]:
openai.organization = ORGID
openai.api_key = APIKEY

### Task 2 - Craft Prompts to Communicate with the API

To communicate with the API we need to learn how to craft a prompt.

A prompt object contains two elements:
1. Role: Specifies the communicator's role—either `User`, `System`, or `Assistant`.
2. Content: Contains the text of the communication

example:
`prompt = {'role': 'user', 'content': 'what is the captial of Italy?'}`

Different Roles:

- **User**: Initiates the conversation, asks questions, and gives instructions to - the AI model.
- **System**: Sets the initial context or instructions for the conversation, guiding the AI's behavior.
- **Assistant**: Generates responses based on the user's queries and the context provided by the system, acting as the AI model's replies.

User initiates the conversation, system provides context, and assistant generates responses.


In [ ]:
user_prompt = {'role': 'user', 'content': 'Name the main four ingrediments of Beef Wellington'}

response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[user_prompt],
    max_tokens=50,
)

content = response['choices'][0]['message']['content']
print('Content: ', content)

### Task 3 - Add context to prompts




To add context to our prompt. We need specify some contents for the `system` role.

In [ ]:
user_prompt = {'role': 'user', 'content': 'Name the main four ingrediments of Beef Wellington'}
system_prompt = {'role': 'system', 'content': 'whatever user asked you, answer with rap lyrics'}

response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[system_prompt, user_prompt],
    max_tokens=100,
)

content = response['choices'][0]['message']['content']
print('Content: ', content)

Exercise: Write a Python function called `prompt` that takes system and user input, sends a request to GPT-4, and returns the response.

In [ ]:
def prompt(system_content, user_content):
  system_prompt = {'role': 'system', 'content': system_content}
  user_prompt = {'role': 'user', 'content': user_content}

  response = openai.ChatCompletion.create(
      model="gpt-4",
      messages=[system_prompt, user_prompt],
      max_tokens=100,
  )

  content = response['choices'][0]['message']['content']
  return content

response_content = prompt(system_content="Whatever user asked you, answer with rap lyrics",
                          user_content="Name the main four ingrediments of Beef Wellington")

print(response_content)

### Task 4 - Include Data in prompts

We have a list of numbers:

In [ ]:
import numpy as np
from statistics import mean, mode, variance

values = [3, 5, 2, 12, 9, 8, 3]
#print(values)
print('Mean: ', mean(values))
#print('Mode: ', mode(values))
#print('Variance: ', variance(values))

Let's create a prompt and include the data in the context


In [ ]:
system_content = f"Answer any question that the user asks you about the following list of values: {values}. Please do not provide any additional explanation."

user_content = "What is the mean of the list?"

response_content = prompt(system_content, user_content)

print(response_content)

Sorting the list: `ascending` and `descending`

In [ ]:
user_content = "Sort the values descending and return th list."

response_content = prompt(system_content, user_content)

print(response_content)

Excercise: Use the prompt method to return only the odd values in the list.

In [ ]:
user_content = "Return only odd values in the list."

response_content = prompt(system_content, user_content)

print(response_content)

### Task 5 - Ask analytical questions based on data


Let's load the `grades` dataset.

In [ ]:
grades = pd.read_csv('grades.csv')
grades.head()

Let's convert the dataframe to json

In [ ]:
json_data = grades.to_json(orient='records')
print(json_data)

Question: Who has the highest grade among the students?

In [ ]:
system_content = f"Answer any question that the user asks you about the following list of values: {json_data}. Please do not provide any additional explanation."
user_content = "Who has the highest grade among the students?"

prompt(system_content, user_content)

Analytical Question: `Based on the data, can we conclude that students who studied more in a week received higher grades?`

In [ ]:
user_content = "Based on the data, can we conclude that students who studied more in a week received higher grades?"

prompt(system_content, user_content)

### Task 6 - Product Sales Analysis with Prompts
1. Loading the `product_sales.csv` dataset
2. Exploring the dataset using prompts
3. Answering some analytical questions

Loading the `product_sales.csv` dataset

In [ ]:
sales = pd.read_csv('product_sales.csv')
json_data = sales.to_json(orient='records')

print(json_data)

Let's explore the dataset

In [ ]:
system_content = f"Act like data analyst. consider the following dataset: {json_data}. Answer the user's questions about the dataset and avoid any additional explanation."
user_content = "Explore the provided product sales data and return a list of 10 bulletpoints containing some facts that can be interesting for data analysts about the data."

response = prompt(system_content, user_content)
print('Response: ', response)

Question 1 - Which product brought the highest Profit?

In [ ]:
user_content = "Which product brought the highest Profit?"

response = prompt(system_content, user_content)
print('Response: ', response)

Let's see if the answer is correct.

In [ ]:
sales.['product_profit'] = (sales.price - sales.cost) * sales.sold_quantity
sales.sort_values(by='product_profit', ascending=False)

Question 2 - Which product category has the lowest product demand?

In [ ]:
user_content = "Which product category has the lowest product demand?"

response = prompt(system_content, user_content)
print('Response: ', response)

ex: Question 3 - Which Product Category brought the highest amount of profit?

In [ ]:
user_content = "Which Product Category brought the highest amount of profit?"

response = prompt(system_content, user_content)
print('Response: ', response)

ex: Write a python code and check if the answer is correct.

In [ ]:
sales.groupby('category')['product_profit'].sum().reset_index().sort_values(by='product_profit', ascending=False)